In [194]:
# Import necessary libraries
import math
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [195]:
df_460users_ratings = pd.read_csv("/content/460users_all_ratings.csv")
df_460users_ratings


,userid,movieid,rating
0,/halloweenhenry/,/film/the-king-of-kong-a-fistful-of-quarters/,10
1,/halloweenhenry/,/film/the-celebration/,7
2,/halloweenhenry/,/film/the-legend-of-hell-house/,8
3,/halloweenhenry/,/film/easy-rider/,8
4,/halloweenhenry/,/film/planet-of-the-apes/,8
...,...,...,...
318423,/jjram00/,/film/werewolf-by-night/,Not Rated
318424,/jjram00/,/film/far-from-the-tree-2021/,Not Rated
318425,/jjram00/,/film/nimic/,Not Rated
318426,/jjram00/,/film/bao-2018/,Not Rated


In [196]:
# Our user's dataframe(user that we will make recommendation)
our_user_df = pd.DataFrame({"userid":[np.nan],"movieid":[np.nan],"rating":[np.nan]}) 
userurl = "/apolaar/" #It should be in /nick/ format.

url="https://letterboxd.com"+userurl+"films/by/rated-date/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
try:
  totalpagenumberstr=soup.find_all("li",attrs={"class":"paginate-page"})[-1].text
  for page in range(int(totalpagenumberstr)):
    urlpage = url+"page/"+str(page+1)

    response = requests.get(urlpage)
    soup = BeautifulSoup(response.content, "html.parser")

    for link in soup.find_all("li",attrs={"class":"poster-container"}):
    #link.div.get("data-film-slug")
    #
      try:
        our_user_df.loc[len(our_user_df.index)]=[userurl,link.div.get("data-target-link"),link.p.span.get("class")[-1].replace("rated-","")]
      # It helps to find all anchor tag’s
      except:
        our_user_df.loc[len(our_user_df.index)]=[userurl,link.div.get("data-target-link"),"Not Rated"]
except:
  print("hello")
our_user_df[our_user_df["rating"]=="icon-16"]="Not Rated but Liked"
our_user_df = our_user_df.iloc[1:,:] #first value comes as nan, we drop it.
our_user_movies = list(our_user_df["movieid"])

In [197]:
df_with_our_user = pd.concat([df_460users_ratings,our_user_df],axis=0)
df_with_our_user = df_with_our_user.query("rating in ['1','2','3','4','5','6','7','8','9','10']")
df_with_our_user

,userid,movieid,rating
0,/halloweenhenry/,/film/the-king-of-kong-a-fistful-of-quarters/,10
1,/halloweenhenry/,/film/the-celebration/,7
2,/halloweenhenry/,/film/the-legend-of-hell-house/,8
3,/halloweenhenry/,/film/easy-rider/,8
4,/halloweenhenry/,/film/planet-of-the-apes/,8
...,...,...,...
406,/apolaar/,/film/stalker/,10
407,/apolaar/,/film/autumn-sonata/,9
408,/apolaar/,/film/wild-strawberries/,9
409,/apolaar/,/film/persona/,10


In [198]:
user_item_matrix

movieid,/film/00-schneider-im-wendekreis-der-eidechse/,/film/0106/,/film/05-mm/,/film/1-2-man/,/film/1-2013-1/,/film/1-54/,/film/1-ichi/,/film/1-knights-in-search-of-the-ravishing-princess-herzelinde/,/film/1-litre-of-tears/,/film/1-mile-to-you/,...,/film/zootopia/,/film/zorba-the-greek/,/film/zorro/,/film/zouzou/,/film/ztracena-brana/,/film/zucchini-cake/,/film/zulu-2013/,/film/zulu/,/film/zygote/,/film/zz-top-that-little-ol-band-from-texas/
userid,,,,,,,,,,,,,,,,,,,,,
/007shubhamgupta/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/4knv/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/4ngei/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/aarestegui/,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/abhishekmenon/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/yellowishrango/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/yeriina/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/yktv/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [199]:
# Drop duplicate entries
df_with_our_user = df_with_our_user.drop_duplicates(['userid', 'movieid'])

df_with_our_user["rating"]=df_with_our_user["rating"].astype(int)
# Create the user-item matrix
user_item_matrix = df_with_our_user.pivot(index='userid', columns='movieid', values='rating').fillna(0)

# Compute the cosine similarity between the user-item matrix
user_similarity = cosine_similarity(user_item_matrix)

# Generate recommendations for each user
def generate_recommendations(user_id, top_n):
    # Get the row index of the user
    user_index = user_item_matrix.index.get_loc(user_id)

    # Get the similarity scores between the user and all other users
    user_scores = user_similarity[user_index]
    
    # Get the indices of the top N similar users
    top_users = user_scores.argsort()[::-1][1:top_n+1]

    # Get the user-item matrix for the top N similar users
    top_users_matrix = user_item_matrix.iloc[top_users]

    # Compute the average rating for each movie across the top N similar users
    top_users_mean = top_users_matrix.mean(axis=0)

    # Get the movies that the user has not rated
    user_movies = user_item_matrix.loc[user_id]
    unrated_movies = user_movies[user_movies==0].index

    # Sort the unrated movies by their average rating across the top N similar users
    sorted_movies = top_users_mean.loc[unrated_movies].sort_values(ascending=False)
    sorted_movies = pd.DataFrame(sorted_movies,columns=["avgrating"])
    sorted_movies = sorted_movies.drop([i for i in our_user_movies if i in sorted_movies.index])
    # Return the top N recommended movies
    return sorted_movies.iloc[:20]

# Generate recommendations for a specific user by finding most similar 3 user.
top_n = 3
recommendations = generate_recommendations(userurl, top_n)

# Print the recommended movies
print("Recommended movies for user {}: \n{}".format(userurl, recommendations))

Recommended movies for user /apolaar/: 
                                                    avgrating
movieid                                                      
/film/paths-of-glory/                                9.666667
/film/dr-strangelove-or-how-i-learned-to-stop-w...   9.333333
/film/la-la-land/                                    9.000000
/film/eraserhead/                                    9.000000
/film/barry-lyndon/                                  9.000000
/film/rear-window/                                   9.000000
/film/three-colors-red/                              9.000000
/film/goodfellas/                                    8.666667
/film/la-dolce-vita/                                 8.666667
/film/blade-runner-2049/                             8.666667
/film/apocalypse-now/                                8.666667
/film/where-is-the-friends-house/                    8.666667
/film/three-colors-blue/                             8.666667
/film/nights-of-cabiria/      